In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
weights_path = '/content/drive/MyDrive/best_model.weights.h5'

In [22]:
from google.colab import drive
import tensorflow as tf

# Mount Google Drive
drive.mount('/content/drive')

def build_shakespeare_model(batch_size=None):
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(None,), batch_size=batch_size),
        tf.keras.layers.Embedding(input_dim=98, output_dim=384),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.GRU(1536, return_sequences=True,
                          dropout=0.3, recurrent_dropout=0.3,
                          stateful=batch_size is not None),
        tf.keras.layers.GRU(768, return_sequences=True,
                          dropout=0.2,
                          stateful=batch_size is not None),
        tf.keras.layers.Dense(98)
    ])
    return model


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
# Initialize for inference (batch_size=1)
model = build_shakespeare_model(batch_size=1)
model.load_weights(weights_path)

In [24]:
# Create test input matching your vocabulary
# Using actual character IDs from your mappings:
test_input = tf.constant([[6, 6, 6, 0, 41, 42, 23, 40, 42, 0]])  # From your sample conversion
print("Test output shape:", model(test_input).shape)  # Should output (1, 10, 98)

Test output shape: (1, 10, 98)


In [26]:
model.save('/content/drive/MyDrive/shakespeare_model.keras')  # New .keras format

# 5. (Optional) Save in HDF5 format if needed
model.save('/content/drive/MyDrive/shakespeare_model.h5')  # Legacy format

In [27]:
# Load the saved model to verify
loaded_model = tf.keras.models.load_model('/content/drive/MyDrive/shakespeare_model.keras')

# Test with sample input
test_input = tf.constant([[6, 6, 6, 0, 41, 42, 23, 40, 42, 0]])
print(loaded_model(test_input).shape)  # Should output (1, 10, 98)

(1, 10, 98)


In [35]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.python.client import device_lib

# Check available accelerators
print("Available devices:")
print(device_lib.list_local_devices())

# Set up multi-GPU if available
strategy = tf.distribute.MirroredStrategy() if len(tf.config.list_physical_devices('GPU')) > 1 else tf.distribute.get_strategy()
print(f"Using strategy: {strategy.__class__.__name__}")

# Download Shakespeare dataset
!mkdir -p data
!wget https://www.gutenberg.org/files/100/100-0.txt -O data/shakespeare.txt
def clean_text(text):
    """More careful cleaning that maintains consistency"""
    text = text.replace('\r', ' ')  # Replace with space instead of removing
    text = text.replace('\n', ' ')  # Replace with space instead of removing
    text = ' '.join(text.split())  # Normalize spaces/
    return text

# 1. Load and clean text
with open('data/shakespeare.txt', 'r', encoding='utf-8') as f:
    text = clean_text(f.read())
print(f"Loaded text length: {len(text)} characters")

# 2. Create vocabulary from CLEANED text
vocab = sorted(set(text))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
vocab_size = len(vocab)

print(f"Vocabulary size: {len(vocab)}")
print("Sample mappings:")
for char in list(vocab)[:5]:
    print(f"'{char}': {char2idx[char]}")

# 3. Convert text to integers
text_as_int = np.array([char2idx[c] for c in text])
print(f"Text as integers shape: {text_as_int.shape}")
print("Sample conversion:", text_as_int[:10])

Available devices:
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15266377498696355182
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14619377664
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5401738242019881787
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]
Using strategy: _DefaultDistributionStrategy
--2025-08-02 18:04:00--  https://www.gutenberg.org/files/100/100-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5618733 (5.4M) [text/plain]
Saving to: ‘data/shakespeare.txt’

data/shakespeare.tx 100%[===================>]   5.36M  9.07MB/s    in 0.6s    

2025-08-02 18:04:01 (9.07 MB/s) - ‘data/shakespeare.txt’ saved [5618733/5618733]

L

In [38]:
# Load and clean text again
def clean_text(text):
    text = text.replace('\r', ' ')
    text = text.replace('\n', ' ')
    text = ' '.join(text.split())
    return text

with open('data/shakespeare.txt', 'r', encoding='utf-8') as f:
    text = clean_text(f.read())


# Recreate vocab
vocab = sorted(set(text))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
vocab_size = len(vocab)


In [43]:
# Define this once in your notebook/script
def reset_rnn_states(model):
    for layer in model.layers:
        if hasattr(layer, 'reset_states'):
            layer.reset_states()


In [44]:
def generate_text(model, start_string, char2idx, idx2char, num_generate=1000, temperature=1.0):
    input_eval = [char2idx[c] for c in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []

    reset_rnn_states(model)  # Reset RNN states properly

    for _ in range(num_generate):
        predictions = model(input_eval)
        predictions = predictions[:, -1, :] / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        text_generated.append(idx2char[predicted_id])
        input_eval = tf.expand_dims([predicted_id], 0)

    return start_string + ''.join(text_generated)



In [47]:
generated = generate_text(
    model=model,
    start_string="What light through yonder window breaks",
    char2idx=char2idx,
    idx2char=idx2char,
    num_generate=1000,
    temperature=0.8  # Lower = more predictable, higher = more creative
)

print(generated)


What light through yonder window breaks, Drawing thy soul to hear what I did never love my head And turn the throat, and therefore no more but such a wife, There are their faults to embrace, and weep no grace and virtue That did begin the seventh of them. But, if I hate, you would not miscontend me much. Thou gav’st thy hand, which cannot be a will. ARVIRAGUS. This is not true; the world’s earthly, And every man that discourse is a shaft between his father’s helm. Troy for’t. A goodly place and give me leave to call it heavy! PRINCESS. Live, That thou wilt swear to your confidence in the perilous clerk, Which, comfortable than the world to come, And you did love as this, but you shall hear. [_Exit._] MENENIUS. Peace, peace! Now plant thee company. Enter Talbot’s Majesty. PROTEUS. They are all abated. A lover I give, and so The glasses of my tongue Give me the edge of an enemy to my life. FALSTAFF. Come, come, neighbour, your bag-a-dog before A heavy head with strength on her sole bount

In [55]:
generated = generate_text(
    model=model,
    start_string="What light through ",
    char2idx=char2idx,
    idx2char=idx2char,
    num_generate=1000,
    temperature=0.8  # Lower = more predictable, higher = more creative
)

print(generated)

What light through y KANI MIDANIWEXAROLONONEx._] ACHENGOLALONGHENINININGERONUSECONIMEx._] IONGROCORYEx._PAVENIVOLANGROXLUREXUSERLINERY**********************************************************************************************************1 IORYENILE: ALORD.) THENOWINOLALORY. wave SACONGONILANÇONFRDUSAUSTHERDYONICOWhenghime._] WARY*********************************** IVALLALATHExeshevithinghig theshilousthe, thar wavis; wild ASTHEWALLALDUSHERONONISCUSTHEx SACORV”5_] m.” t thanghe._] way MOLUSTHENINILLONÇOLLATHERY._] NISENIONOLANIVANGONGONILLELANONI whe?_] ICANO._ERDUSExiz: LANEFIVICOWELLORDARIO, owe85 wizèSTHERO._] THEWAGRONGEBONIOLLONISTLYESTHUIVEBENExcheg! ALALLDIVIVERONGONGEx, he, THENINONICOWALLANINGONCLALSHENICOBELACONDENGHENILUSHEx THEDADALURLNGONÇONENANEx, thinghavinghenil WENISCENENIUSHENIORDALLIVEx._] LARY.” way? prdspld!”œ chizèd muld MALIONGHExe; MOWI THENGHUSILAUCONINI_] f ANIOLLACENILLAVICOLANGONOVAYONGRYOGEx ALIVExesby.” thevechere ACONICONICKICONGUSHANGUSANENIVENILLYONGH

In [56]:
generated = generate_text(
    model=model,
    start_string="i love her",
    char2idx=char2idx,
    idx2char=idx2char,
    num_generate=1000,
    temperature=0.8  # Lower = more predictable, higher = more creative
)

print(generated)

i love her! CONGEX._] watheng; bysthe winghe wixenghed!” d MONGERDATHExigavequ ALATHELUSCERORLOLANICONGERY._ONGRY”) THExprethe? wathy, thavemaraclove8 SCONGONEXEROWAUFRENE._] t sothe ANGONISHENILUSHENI.” Whemize; KINGENIVENINÇONÇONUSCOXENICOLOLAFRY.” chabjERILALLEBESTHENIVERUSHALALONGExing, ANIONGENIOLALENIVEXERY CONILUSHUSHEx akere t k!_AMANGONICAMENIATHORDRATHALONROLOLOLALIONEKèd ALALLLLLUSCOLARORONIONÇENI THENGLALLUSENINWALOPHERI TERDECONÇOWADIOLURORDOLLOWESENINIVENIONENULURUSTHENIDUSEXEx; wake; musthowhequghachet, ivesthak’s. che fONGONIORONIONIVO2] TONGHUSCANULONGENÇONGUSTHERYONDANILIOVEXALORONÇONILONGLANIOLAGORYONTHERO: why, FIVIOLOXENGHESHETHELLSWIVECONGEXAVENICANICODAGONERY. VRONICKIONILONENXENINVADALONORA…” ICANIONESANEXTHUCOLLORONILADExure—flà” BALLY***************************** THURIVENGLUMO. ghœ feÇESTHExiz] bene—LANONIORDGORLANICOLLONOXENENGLANGOLOLANUSTLAN6 WIONVENONILICAFONIOCLONILLURDANILOWANIONI WADENÇOLOUCOLY.) THERDExesthevesure. ANILUSExet mangouxeÉ. wourard BEx THE

In [48]:
# Reload text and mappings
def clean_text(text):
    text = text.replace('\r', ' ')
    text = text.replace('\n', ' ')
    text = ' '.join(text.split())
    return text

with open('data/shakespeare.txt', 'r', encoding='utf-8') as f:
    text = clean_text(f.read())

vocab = sorted(set(text))
char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

# Dataset parameters
SEQ_LENGTH = 100
BATCH_SIZE = 128
BUFFER_SIZE = 10000

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

# Build dataset
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(SEQ_LENGTH + 1, drop_remainder=True)
dataset = sequences.map(split_input_target)
full_dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Split into training and validation
dataset_size = len(list(full_dataset))
train_size = int(0.8 * dataset_size)
val_dataset = full_dataset.skip(train_size).prefetch(tf.data.AUTOTUNE)


In [52]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005, clipnorm=1.0),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)



In [54]:
# Rebuild the model for flexible batch size
model = build_shakespeare_model(batch_size=None)  # Not stateful
model.load_weights('/content/drive/MyDrive/best_model.weights.h5')

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005, clipnorm=1.0),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

loss, accuracy = model.evaluate(val_dataset)
print(f"✅ Validation Loss: {loss:.4f}")
print(f"✅ Validation Accuracy: {accuracy:.4f}")


82/82 ━━━━━━━━━━━━━━━━━━━━ 25s 168ms/step - accuracy: 0.6272 - loss: 1.1987
✅ Validation Loss: 1.2009
✅ Validation Accuracy: 0.6264


In [29]:
# 1. Character vocabulary (recreate from training)
vocab = sorted(list(set(
    ' !"#$%&\'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwxyz{|}~\n\t'
)))  # Adjust this if needed

char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

# 2. Convert input text to integer sequence
def text_to_input_tensor(text):
    input_ids = [char2idx[c] for c in text if c in char2idx]
    return tf.expand_dims(input_ids, 0)  # shape (1, len)

# 3. Get model prediction
def predict_next_char(model, input_text, temperature=1.0):
    input_tensor = text_to_input_tensor(input_text)
    prediction = model(input_tensor)
    prediction = prediction[:, -1, :]  # last character's prediction
    prediction = prediction / temperature
    predicted_id = tf.random.categorical(prediction, num_samples=1)[-1, 0].numpy()
    return idx2char[predicted_id]

# 4. Generate text from seed
def generate_text(model, start_string, num_generate=300, temperature=1.0):
    input_eval = [char2idx[c] for c in start_string if c in char2idx]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = list(start_string)

    model.reset_states()
    for _ in range(num_generate):
        predictions = model(input_eval)
        predictions = predictions[:, -1, :] / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        text_generated.append(idx2char[predicted_id])
        input_eval = tf.expand_dims([predicted_id], 0)

    return ''.join(text_generated)


In [30]:
print("Next char after 'ROMEO':", predict_next_char(model, "ROMEO"))

generated_text = generate_text(model, "ROMEO: ", num_generate=400, temperature=0.8)
print(generated_text)


Next char after 'ROMEO': Q


AttributeError: 'Sequential' object has no attribute 'reset_states'